In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import RandomForest
from time import *
from pyspark.mllib.evaluation import BinaryClassificationMetrics

In [10]:
CSV_PATH = "/user/spark/share/Final_All1022.csv"
APP_NAME = "bdse1122"
SPARK_URL = "yarn"
RANDOM_SEED= 13579
TRAINING_DATA_RATIO= 0.7
RF_NUM_TREES= 3
RF_MAX_DEPTH= 4
RF_MAX_BINS= 32

In [24]:
spark = SparkSession.builder \
    .appName(APP_NAME) \
    .master(SPARK_URL) \
    .config('spark.executor.instances','99')\
    .getOrCreate()

In [25]:
df = spark.read \
    .options(header = "true", inferschema = "true",sep=',',multiLine=True) \
    .csv(CSV_PATH)

In [14]:
df.head()

Row(日期=datetime.datetime(2012, 5, 2, 0, 0), 證券代號=1471, 產業類別='電子工業', 證券名稱='首利', 成交股數=810201.0, 成交筆數=350.0, 成交金額=9890836.0, 開盤價=11.65, 最高價=12.4, 最低價=11.6, 收盤價=12.4, 漲跌價差=0.8, 當日股價變化(%)=0.06437768240343347, 當日股價最大震幅(%)=0.06866952789699576, 漲/跌=1.0, 股價5日移動平均=11.869999999999987, 股價10日移動平均=11.755, 股價30日移動平均=12.35833333333326, 股價90日移動平均=12.378888888888916, 外資買賣超股數=41000.0, 投信買賣超股數=0.0, 自營商買賣超股數=0.0, 三大法人買賣超股數=41000.0, 本益比=-1.0, 殖利率(%)=0.0, 股價淨值比=0.89, TW_Industrial_production_norm=-0.4919657045682684, TW_manufacturing_norm=0.3815616443190921, TW_Export_value_norm=-0.5550970900658893, TW_Import_value_norm=-0.7848240661927902, TW_Consumer_price_norm=-1.542645474716008, TW_Industrial_production_maxmin=0.43294379521424603, TW_manufacturing_maxmin=0.5425531914893618, TW_Export_value_maxmin=0.2956693778852495, TW_Import_value_maxmin=0.22165820642977999, TW_Consumer_price_maxmin=0.18686868686868555, Open_USD_norm=-0.8482300776892254, High_USD_norm=-0.8651667686077162, Low_USD_norm=-0.833733103783761

In [26]:
transformed_df = df.rdd.map(lambda row: LabeledPoint(row[-1], Vectors.dense(row[0:-1])))

splits = [TRAINING_DATA_RATIO, 1.0 - TRAINING_DATA_RATIO]
training_data, test_data = transformed_df.randomSplit(splits, RANDOM_SEED)

In [28]:
start_time = time()

model = RandomForest.trainClassifier(training_data, numClasses=2, categoricalFeaturesInfo={},numTrees=RF_NUM_TREES, featureSubsetStrategy="auto", impurity="gini",maxDepth=RF_MAX_DEPTH, maxBins=RF_MAX_BINS, seed=RANDOM_SEED)

end_time = time()
elapsed_time = end_time - start_time
print("Time to train model: %.3f seconds" % elapsed_time)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 4 times, most recent failure: Lost task 0.3 in stage 3.0 (TID 9, bdse58.example.org, executor 17): java.io.IOException: Cannot run program "python": error=2, No such file or directory
	at java.lang.ProcessBuilder.start(ProcessBuilder.java:1048)
	at org.apache.spark.api.python.PythonWorkerFactory.startDaemon(PythonWorkerFactory.scala:197)
	at org.apache.spark.api.python.PythonWorkerFactory.createThroughDaemon(PythonWorkerFactory.scala:122)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:95)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:117)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:109)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.

In [ ]:
predictions = model.predict(test_data.map(lambda x: x.features))
labels_and_predictions = test_data.map(lambda x: x.label).zip(predictions)
acc = labels_and_predictions.filter(lambda x: x[0] == x[1]).count() / float(test_data.count())
print("Model accuracy: %.3f%%" % (acc * 100))

In [ ]:
start_time = time()

metrics = BinaryClassificationMetrics(labels_and_predictions)
print("Area under Precision/Recall (PR) curve: %.f" % (metrics.areaUnderPR * 100))
print("Area under Receiver Operating Characteristic (ROC) curve: %.3f" % (metrics.areaUnderROC * 100))

end_time = time()
elapsed_time = end_time - start_time
print("Time to evaluate model: %.3f seconds" % elapsed_time)

In [23]:
spark.stop()